In [3]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Roraima - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [7]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_RR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Roraima - Produção de Cimento (t)', axis=1)
data

,Ano,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - IDH,Roraima - IDH Educacao,Roraima - Consumo de Cimento (t)
0,2014,7.349320e+06,344289.736627,12.154911,5.667961e+06,10.328300,0.716277,5.676413e+08,2.779340e+09,1.954361e+10,...,14.644773,1001.940536,1542.131372,27.986329,20.647922,66.896135,1.956684,0.735427,0.734250,120.417571
1,2015,7.439258e+06,340498.356490,12.130631,5.685185e+06,10.723767,0.703535,5.179392e+08,2.561731e+09,1.327596e+10,...,19.007683,1001.988306,1541.272845,28.048825,20.665808,66.738045,1.910770,0.736260,0.740251,113.282057
2,2016,7.513614e+06,340512.038701,12.120014,5.705074e+06,10.764101,0.702129,4.524144e+08,2.516859e+09,1.011370e+10,...,24.437747,1002.055065,1514.905960,27.959130,20.665948,67.018448,1.902067,0.735193,0.738638,105.481371
3,2017,7.560265e+06,340005.357917,12.115483,5.723617e+06,10.533542,0.701049,4.213689e+08,2.497710e+09,8.456309e+09,...,26.318142,1002.155784,1495.611624,27.867978,20.547545,66.903661,1.943183,0.733665,0.735696,99.723000
4,2018,7.565391e+06,340384.545692,12.089988,5.726485e+06,10.642202,0.701418,4.166580e+08,2.797145e+09,7.937760e+09,...,28.548818,1002.183834,1468.719756,27.821635,20.578691,67.224766,1.912675,0.732543,0.733123,83.012000
5,2019,7.519467e+06,340174.819510,12.077845,5.712961e+06,10.430272,0.699456,4.462625e+08,3.484252e+09,1.091282e+10,...,24.615921,1001.746371,1214.342104,28.427536,20.385092,64.243880,1.959458,0.730047,0.729406,121.120000
6,2020,7.427922e+06,340776.584854,12.078197,5.688956e+06,9.898626,0.699770,5.489412e+08,4.114911e+09,1.625333e+10,...,15.236170,1001.355351,881.861438,29.108198,20.010993,59.875634,2.142193,0.728324,0.725989,151.571000
7,2021,7.252613e+06,340641.391154,12.095909,5.643384e+06,9.503368,0.701567,6.638811e+08,4.421848e+09,1.933031e+10,...,9.673941,1001.344782,867.033434,28.876218,19.963349,59.298235,2.217953,0.727354,0.722988,144.989000
8,2022,7.041849e+06,340251.126772,12.115271,5.587543e+06,9.643865,0.703528,7.010332e+08,4.533216e+09,1.997888e+10,...,9.037557,1001.455310,994.638947,28.471336,20.091016,60.304102,2.178437,0.729932,0.727532,132.217000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Roraima - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - IDH,Roraima - IDH Educacao
0,7.349320e+06,344289.736627,12.154911,5.667961e+06,10.328300,0.716277,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,0.757094,14.644773,1001.940536,1542.131372,27.986329,20.647922,66.896135,1.956684,0.735427,0.734250
1,7.439258e+06,340498.356490,12.130631,5.685185e+06,10.723767,0.703535,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,0.766989,19.007683,1001.988306,1541.272845,28.048825,20.665808,66.738045,1.910770,0.736260,0.740251
2,7.513614e+06,340512.038701,12.120014,5.705074e+06,10.764101,0.702129,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,0.766897,24.437747,1002.055065,1514.905960,27.959130,20.665948,67.018448,1.902067,0.735193,0.738638
3,7.560265e+06,340005.357917,12.115483,5.723617e+06,10.533542,0.701049,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,0.766251,26.318142,1002.155784,1495.611624,27.867978,20.547545,66.903661,1.943183,0.733665,0.735696
4,7.565391e+06,340384.545692,12.089988,5.726485e+06,10.642202,0.701418,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,0.765092,28.548818,1002.183834,1468.719756,27.821635,20.578691,67.224766,1.912675,0.732543,0.733123
5,7.519467e+06,340174.819510,12.077845,5.712961e+06,10.430272,0.699456,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,0.763373,24.615921,1001.746371,1214.342104,28.427536,20.385092,64.243880,1.959458,0.730047,0.729406
6,7.427922e+06,340776.584854,12.078197,5.688956e+06,9.898626,0.699770,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,0.761135,15.236170,1001.355351,881.861438,29.108198,20.010993,59.875634,2.142193,0.728324,0.725989


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    113.282057
1    105.481371
2     99.723000
3     83.012000
4    121.120000
5    151.571000
6    144.989000
7    132.217000
8           NaN
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Roraima - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - IDH,Roraima - IDH Educacao
0,-1.873932,2.219872,1.579198,-1.701846,-1.552820,2.183061,1.779974,0.019134,2.003355,-1.880579,...,-2.086944,-1.764862,-0.486347,0.695472,-0.163201,0.669938,0.383734,1.120559,0.750536,-0.274054
1,-0.686201,-0.321105,0.623000,-0.878147,0.984068,-0.078539,0.870373,-0.621387,0.401153,0.111653,...,0.785615,-0.835376,-0.159647,0.687942,0.153123,0.851259,0.228970,-0.867625,1.148268,1.408315
2,0.295747,-0.311935,0.204878,0.073013,1.242807,-0.328079,-0.328802,-0.753466,-0.407217,1.154682,...,0.758928,0.321460,0.296921,0.456703,-0.300866,0.852671,0.503476,-1.244456,0.638501,0.956020
3,0.911825,-0.651512,0.026437,0.959829,-0.236209,-0.519627,-0.896969,-0.809829,-0.830898,0.955301,...,0.571459,0.722065,0.985738,0.287491,-0.762234,-0.347595,0.391103,0.535952,-0.091293,0.131325
4,0.979518,-0.397381,-0.977650,1.096946,0.460835,-0.454288,-0.983185,0.071542,-0.963455,0.100493,...,0.234984,1.197294,1.177573,0.051648,-0.996797,-0.031868,0.705454,-0.785104,-0.626904,-0.589959
5,0.373043,-0.537939,-1.455862,0.450205,-0.898681,-0.802528,-0.441390,2.094006,-0.202939,-0.441549,...,-0.264042,0.359419,-1.814238,-2.179256,2.069974,-1.994406,-2.212737,1.240674,-1.819107,-1.631647


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    113.282057
1    105.481371
2     99.723000
3     83.012000
4    121.120000
5    151.571000
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Roraima - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - IDH,Roraima - IDH Educacao
6,-0.737944,-0.124488,-1.172235,-0.624692,-2.087431,-0.66538,1.16956,2.035192,0.985175,-0.705937,...,-0.799822,-1.289882,-2.110168,-2.173877,2.208509,-2.227691,-2.268205,2.353163,-1.73084,-1.713392


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    144.989
Name: Roraima - Consumo de Cimento (t), dtype: float64

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3761269757, 2797400073, 687614046, 60251005, 1770294061, 4148879401, 2313734447, 1254618, 1168688150, 1060593154]


Step: 0 ___________________________________________


2023-08-20 12:01:26.019817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-20 12:01:26.020699: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 3430: early stopping
val_loss: 45.741065979003906
winner_seed: 3761269757


Step: 1 ___________________________________________
Epoch 1402: early stopping
val_loss: 183.54469299316406


Step: 2 ___________________________________________
Epoch 2218: early stopping
val_loss: 231.61080932617188


Step: 3 ___________________________________________
Epoch 1288: early stopping
val_loss: 434.1674499511719


Step: 4 ___________________________________________
Epoch 1915: early stopping
val_loss: 810.9033203125


Step: 5 ___________________________________________
Epoch 3504: early stopping
val_loss: 231.82078552246094


Step: 6 ___________________________________________
Epoch 2267: early stopping
val_loss: 169.39695739746094


Step: 7 ___________________________________________
Epoch 2127: early stopping
val_loss: 8.405186235904694e-08
winner_seed: 1254618


Step: 8 ___________________________________________
Epoch 1887: early stopping
val_loss: 124.39417266845703


Step: 9 ___________

In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 488ms/step - loss: 13053.2256 - val_loss: 20220.4043
Epoch 2/5000
1/1 [==============================] - 0s 33ms/step - loss: 11926.0049 - val_loss: 35234.5547
Epoch 3/5000
1/1 [==============================] - 0s 35ms/step - loss: 12741.9258 - val_loss: 28163.4824
Epoch 4/5000
1/1 [==============================] - 0s 34ms/step - loss: 12562.8438 - val_loss: 15157.3193
Epoch 5/5000
1/1 [==============================] - 0s 36ms/step - loss: 10981.4248 - val_loss: 15808.4902
Epoch 6/5000
1/1 [==============================] - 0s 37ms/step - loss: 10521.2500 - val_loss: 15223.9961
Epoch 7/5000
1/1 [==============================] - 0s 38ms/step - loss: 10046.0088 - val_loss: 14937.6172
Epoch 8/5000
1/1 [==============================] - 0s 37ms/step - loss: 9736.6572 - val_loss: 13984.4482
Epoch 9/5000
1/1 [==============================] - 0s 41ms/step - loss: 9356.0977 - val_loss: 13666.1201
Epoch 10/5000
1/1 [===================

1/1 [==============================] - 0s 38ms/step - loss: 746.4050 - val_loss: 2134.4128
Epoch 79/5000
1/1 [==============================] - 0s 36ms/step - loss: 724.0466 - val_loss: 2091.6414
Epoch 80/5000
1/1 [==============================] - 0s 38ms/step - loss: 702.5730 - val_loss: 2050.0967
Epoch 81/5000
1/1 [==============================] - 0s 43ms/step - loss: 681.9488 - val_loss: 2009.7728
Epoch 82/5000
1/1 [==============================] - 0s 38ms/step - loss: 662.1424 - val_loss: 1970.6692
Epoch 83/5000
1/1 [==============================] - 0s 40ms/step - loss: 643.1202 - val_loss: 1932.7123
Epoch 84/5000
1/1 [==============================] - 0s 37ms/step - loss: 624.8508 - val_loss: 1895.8739
Epoch 85/5000
1/1 [==============================] - 0s 33ms/step - loss: 607.3057 - val_loss: 1860.1157
Epoch 86/5000
1/1 [==============================] - 0s 35ms/step - loss: 590.4551 - val_loss: 1825.4053
Epoch 87/5000
1/1 [==============================] - 0s 34ms/step - l

Epoch 156/5000
1/1 [==============================] - 0s 34ms/step - loss: 205.9360 - val_loss: 768.9586
Epoch 157/5000
1/1 [==============================] - 0s 35ms/step - loss: 204.9789 - val_loss: 763.6136
Epoch 158/5000
1/1 [==============================] - 0s 34ms/step - loss: 204.0603 - val_loss: 758.4084
Epoch 159/5000
1/1 [==============================] - 0s 34ms/step - loss: 203.1779 - val_loss: 753.3197
Epoch 160/5000
1/1 [==============================] - 0s 36ms/step - loss: 202.3309 - val_loss: 748.3517
Epoch 161/5000
1/1 [==============================] - 0s 34ms/step - loss: 201.5170 - val_loss: 743.4979
Epoch 162/5000
1/1 [==============================] - 0s 36ms/step - loss: 200.7350 - val_loss: 738.7570
Epoch 163/5000
1/1 [==============================] - 0s 36ms/step - loss: 199.9852 - val_loss: 734.1263
Epoch 164/5000
1/1 [==============================] - 0s 35ms/step - loss: 199.2642 - val_loss: 729.6016
Epoch 165/5000
1/1 [==============================] - 0

1/1 [==============================] - 0s 34ms/step - loss: 182.8135 - val_loss: 571.6686
Epoch 235/5000
1/1 [==============================] - 0s 34ms/step - loss: 182.7721 - val_loss: 570.7186
Epoch 236/5000
1/1 [==============================] - 0s 32ms/step - loss: 182.7330 - val_loss: 569.7847
Epoch 237/5000
1/1 [==============================] - 0s 34ms/step - loss: 182.6950 - val_loss: 568.8719
Epoch 238/5000
1/1 [==============================] - 0s 35ms/step - loss: 182.6589 - val_loss: 567.9781
Epoch 239/5000
1/1 [==============================] - 0s 32ms/step - loss: 182.6247 - val_loss: 567.1220
Epoch 240/5000
1/1 [==============================] - 0s 32ms/step - loss: 182.5903 - val_loss: 566.2375
Epoch 241/5000
1/1 [==============================] - 0s 41ms/step - loss: 182.5584 - val_loss: 565.4063
Epoch 242/5000
1/1 [==============================] - 0s 37ms/step - loss: 182.5275 - val_loss: 564.5805
Epoch 243/5000
1/1 [==============================] - 0s 35ms/step - l

1/1 [==============================] - 0s 33ms/step - loss: 181.8239 - val_loss: 534.5123
Epoch 313/5000
1/1 [==============================] - 0s 36ms/step - loss: 181.8220 - val_loss: 534.3239
Epoch 314/5000
1/1 [==============================] - 0s 34ms/step - loss: 181.8205 - val_loss: 534.1342
Epoch 315/5000
1/1 [==============================] - 0s 39ms/step - loss: 181.8195 - val_loss: 533.9565
Epoch 316/5000
1/1 [==============================] - 0s 38ms/step - loss: 181.8167 - val_loss: 533.7640
Epoch 317/5000
1/1 [==============================] - 0s 33ms/step - loss: 181.8157 - val_loss: 533.5983
Epoch 318/5000
1/1 [==============================] - 0s 33ms/step - loss: 181.8142 - val_loss: 533.4228
Epoch 319/5000
1/1 [==============================] - 0s 41ms/step - loss: 181.8127 - val_loss: 533.2551
Epoch 320/5000
1/1 [==============================] - 0s 37ms/step - loss: 181.8115 - val_loss: 533.0894
Epoch 321/5000
1/1 [==============================] - 0s 39ms/step - l

...

1/1 [==============================] - 0s 35ms/step - loss: 92.7170 - val_loss: 228.2434
Epoch 1813/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7173 - val_loss: 228.4551
Epoch 1814/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7165 - val_loss: 228.2969
Epoch 1815/5000
1/1 [==============================] - 0s 35ms/step - loss: 92.7162 - val_loss: 228.3638
Epoch 1816/5000
1/1 [==============================] - 0s 37ms/step - loss: 92.7161 - val_loss: 228.4371
Epoch 1817/5000
1/1 [==============================] - 0s 38ms/step - loss: 92.7157 - val_loss: 228.4828
Epoch 1818/5000
1/1 [==============================] - 0s 35ms/step - loss: 92.7156 - val_loss: 228.4938
Epoch 1819/5000
1/1 [==============================] - 0s 39ms/step - loss: 92.7154 - val_loss: 228.5921
Epoch 1820/5000
1/1 [==============================] - 0s 33ms/step - loss: 92.7152 - val_loss: 228.6327
Epoch 1821/5000
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 92.7093 - val_loss: 230.7607
Epoch 1891/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7093 - val_loss: 230.7737
Epoch 1892/5000
1/1 [==============================] - 0s 32ms/step - loss: 92.7132 - val_loss: 231.3564
Epoch 1893/5000
1/1 [==============================] - 0s 35ms/step - loss: 92.7096 - val_loss: 230.6319
Epoch 1894/5000
1/1 [==============================] - 0s 35ms/step - loss: 92.7093 - val_loss: 230.7756
Epoch 1895/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7092 - val_loss: 230.7821
Epoch 1896/5000
1/1 [==============================] - 0s 33ms/step - loss: 92.7091 - val_loss: 230.8057
Epoch 1897/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7093 - val_loss: 230.8284
Epoch 1898/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7092 - val_loss: 230.8410
Epoch 1899/5000
1/1 [==============================] - 0s 34ms/step - l

1/1 [==============================] - 0s 34ms/step - loss: 92.7085 - val_loss: 231.4924
Epoch 1969/5000
1/1 [==============================] - 0s 36ms/step - loss: 92.7084 - val_loss: 231.5077
Epoch 1970/5000
1/1 [==============================] - 0s 40ms/step - loss: 92.7084 - val_loss: 231.5100
Epoch 1971/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7082 - val_loss: 231.5258
Epoch 1972/5000
1/1 [==============================] - 0s 39ms/step - loss: 92.7086 - val_loss: 231.5133
Epoch 1973/5000
1/1 [==============================] - 0s 43ms/step - loss: 92.7085 - val_loss: 231.5286
Epoch 1974/5000
1/1 [==============================] - 0s 43ms/step - loss: 92.7084 - val_loss: 231.5277
Epoch 1975/5000
1/1 [==============================] - 0s 33ms/step - loss: 92.7083 - val_loss: 231.5319
Epoch 1976/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7085 - val_loss: 231.5402
Epoch 1977/5000
1/1 [==============================] - 0s 36ms/step - l

1/1 [==============================] - 0s 34ms/step - loss: 92.7076 - val_loss: 231.7418
Epoch 2047/5000
1/1 [==============================] - 0s 37ms/step - loss: 92.7073 - val_loss: 231.7302
Epoch 2048/5000
1/1 [==============================] - 0s 36ms/step - loss: 92.7076 - val_loss: 231.7353
Epoch 2049/5000
1/1 [==============================] - 0s 38ms/step - loss: 92.7078 - val_loss: 231.7311
Epoch 2050/5000
1/1 [==============================] - 0s 40ms/step - loss: 92.7074 - val_loss: 231.7344
Epoch 2051/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7069 - val_loss: 231.7334
Epoch 2052/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7078 - val_loss: 231.6466
Epoch 2053/5000
1/1 [==============================] - 0s 35ms/step - loss: 92.7076 - val_loss: 231.7618
Epoch 2054/5000
1/1 [==============================] - 0s 34ms/step - loss: 92.7072 - val_loss: 231.6749
Epoch 2055/5000
1/1 [==============================] - 0s 35ms/step - l

1/1 [==============================] - 0s 34ms/step - loss: 75.8101 - val_loss: 183.3897
Epoch 2125/5000
1/1 [==============================] - 0s 35ms/step - loss: 74.4170 - val_loss: 185.0303
Epoch 2126/5000
1/1 [==============================] - 0s 35ms/step - loss: 72.9003 - val_loss: 174.3314
Epoch 2127/5000
1/1 [==============================] - 0s 39ms/step - loss: 71.2115 - val_loss: 188.8096
Epoch 2127: early stopping


In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 66ms/step
[144.989](test_target) - [[137.8302]](prediction) = 7.158799804687504


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Roraima - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - IDH,Roraima - IDH Educacao
0,-1.261575,1.414207,1.350510,-1.194400,-1.409246,1.408450,1.165029,1.396038,1.335266,-1.330895,...,-1.414167,-1.177929,-1.151791,0.740804,-0.313330,-1.414182,0.103893,1.398581,-0.435379,-1.365672
1,0.077325,-0.710938,-0.311809,-0.058586,0.602056,-0.593765,0.111769,-0.502296,-0.264151,0.251276,...,0.716983,-0.088799,-0.134762,0.672865,1.350972,0.698864,-1.273382,-0.517690,1.382951,1.000964
2,1.184251,-0.703269,-1.038701,1.252987,0.807190,-0.814685,-1.276798,-0.893743,-1.071116,1.079619,...,0.697184,1.266728,1.286553,-1.413669,-1.037642,0.715317,1.169489,-0.880891,-0.947572,0.364708


train_target:


,Roraima - Consumo de Cimento (t)
0,113.282057
1,105.481371
2,99.723000


test_input:


...

,Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Roraima - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Roraima - IDH,Roraima - IDH Educacao
6,-0.737944,-0.124488,-1.172235,-0.624692,-2.087431,-0.66538,1.16956,2.035192,0.985175,-0.705937,...,-0.799822,-1.289882,-2.110168,-2.173877,2.208509,-2.227691,-2.268205,2.353163,-1.73084,-1.713392


test_target:


,Roraima - Consumo de Cimento (t)
6,144.989


Epoch 2127: early stopping
1/1 [==============================] - 0s 34ms/step
Error: 7.158799804687504




[102.60204315185547, 91.36731719970703, 113.28311157226562, 137.8302001953125]

In [21]:
display(targets)
display(predictions)

[83.012, 121.12, 151.571, 144.989]

[102.60204315185547, 91.36731719970703, 113.28311157226562, 137.8302001953125]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

23.69735354614258

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.18931681389870483